In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [3]:
!wget --no-check-certificate \
    "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip" \
    -O "/tmp/cats_and_dogs.zip"

import zipfile
with zipfile.ZipFile("/tmp/cats_and_dogs.zip", 'r') as zip_ref:
    zip_ref.extractall("/tmp/cats_and_dogs")

--2025-02-14 06:51:10--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 142.250.141.207, 74.125.137.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs.zip’

/tmp/cats_and_dogs. 100%[===================>]  65.43M   207MB/s    in 0.3s    

2025-02-14 06:51:10 (207 MB/s) - ‘/tmp/cats_and_dogs.zip’ saved [68606236/68606236]



In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    '/tmp/cats_and_dogs/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/tmp/cats_and_dogs/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 2400 images belonging to 1 classes.
Found 600 images belonging to 1 classes.


In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Congele as camadas da base_model
for layer in base_model.layers:
    layer.trainable = False

# Crie o modelo
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5
)

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


75/75 ━━━━━━━━━━━━━━━━━━━━ 673s 9s/step - accuracy: 0.9697 - loss: 0.0435 - val_accuracy: 1.0000 - val_loss: 1.9440e-27
Epoch 2/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 742s 10s/step - accuracy: 1.0000 - loss: 1.6384e-26 - val_accuracy: 1.0000 - val_loss: 1.8678e-27
Epoch 3/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 681s 9s/step - accuracy: 1.0000 - loss: 3.1986e-26 - val_accuracy: 1.0000 - val_loss: 1.8781e-27
Epoch 4/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 682s 9s/step - accuracy: 1.0000 - loss: 2.9817e-26 - val_accuracy: 1.0000 - val_loss: 1.8782e-27
Epoch 5/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 667s 9s/step - accuracy: 1.0000 - loss: 9.9944e-27 - val_accuracy: 1.0000 - val_loss: 1.8781e-27


In [7]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy:.2f}')

19/19 ━━━━━━━━━━━━━━━━━━━━ 134s 7s/step - accuracy: 1.0000 - loss: 2.9570e-27
Validation accuracy: 1.00


In [8]:
model.save('cat_dog_model.h5')